# **Python Script to scrape recipes from multiple sources to create a dataset** 

### **dataset is named as "recipes.csv".**

## **Dataset Sources**
[BBC Good Food](https://www.bbcgoodfood.com/)

[Spark Recipes](https://recipes.sparkpeople.com/)

***Created by P. Akshay Kumar***

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json

In [ ]:
#scrape URLs of recipes
meals=["breakfast","lunch","dinner"]
pages=[11,20,26]
for i,meal in enumerate(meals):
    url_list=[]
    for j in range(pages[i]):
        print(meal,j+1)
        html=requests.get("https://www.bbcgoodfood.com/search/recipes/page/{}?q={}&sort=-relevance".format(j+1,meal)).text
        soup = BeautifulSoup(html,"html.parser")
        result=soup.find_all('a',{"class":"standard-card-new__article-title"})
        for i in range(0,len(result)): 
           url_list.append(result[i]['href'])
    with open('{}_url_list.txt'.format(meal), 'w') as f: 
       for item in url_list: 
           f.write("%s\n" % item)
    f.close()

breakfast 1
breakfast 2
breakfast 3
breakfast 4
breakfast 5
breakfast 6
breakfast 7
breakfast 8
breakfast 9
breakfast 10
breakfast 11
lunch 1
lunch 2
lunch 3
lunch 4
lunch 5
lunch 6
lunch 7
lunch 8
lunch 9
lunch 10
lunch 11
lunch 12
lunch 13
lunch 14
lunch 15
lunch 16
lunch 17
lunch 18
lunch 19
lunch 20
dinner 1
dinner 2
dinner 3
dinner 4
dinner 5
dinner 6
dinner 7
dinner 8
dinner 9
dinner 10
dinner 11
dinner 12
dinner 13
dinner 14
dinner 15
dinner 16
dinner 17
dinner 18
dinner 19
dinner 20
dinner 21
dinner 22
dinner 23
dinner 24
dinner 25
dinner 26


In [ ]:
meal_url_list={}
for meal in meals:
    with open('{}_url_list.txt'.format(meal), 'r') as f: 
       data=f.read()
    url_list=data.split()
    meal_url_list.setdefault(meal,[])
    meal_url_list[meal]=url_list

In [ ]:
final_recipes=[]
recipe_count=1
for meal in meals:
    for index,url in enumerate(meal_url_list[meal]): 
        recipe=[]
        recipe_ingredient_dict={}
        recipe_ingredient_dict.setdefault("Recipe {}".format(recipe_count),[]) 
        url="https://www.bbcgoodfood.com"+url
        #recipe_ingredient_dict.setdefault(index+1,[]) 
        #Scrape ingredients of recipe 
        html=requests.get(url).text 
        soup = BeautifulSoup(html,"html.parser") 
        result=soup.find_all('li',{"class":"pb-xxs pt-xxs list-item list-item--separator"}) 

        print("Recipe {}: {}".format(recipe_count,url))
        
        recipe_id={}
        recipe_id["Title"]=url.split('/')[-1].replace('-',' ').title()

        recipe.append(recipe_id)

        #recipe_dict["RECIPE {}".format(index+1)]=url.split('/')[-1].replace('-',' ').title()
        #f.write("RECIPE {}: {}\n\n".format(index+1,url.split('/')[-1].replace('-',' ').title())) 

        #print("INGREDIENTS SECTION\n") 
        #f.write("INGREDIENTS SECTION\n\n")
        
        url_dict={}
        url_dict["URL"]=url
        recipe.append(url_dict)
        
        meal_dict={}
        meal_dict["Meal"]=meal
        recipe.append(meal_dict)
        
        rating=soup.find_all('span',{"class":"sr-only"},text=re.compile("Rating.*"))
        if len(rating)>0:
            rating=rating[0].text.split(":")[1]
        else:
            rating=None

        num_of_rating=None
        num_of_ratings=soup.find_all('span',{"class":"rating__count-text body-copy-small"})
        if len(num_of_ratings)>0:
            num_of_ratings=num_of_ratings[0].text
        else:
            num_of_ratings=None
        
        ratings_dict={}
        if "ratings" in num_of_ratings:
            num_of_ratings=re.sub("ratings","",num_of_ratings).strip()
        elif "rating" in num_of_ratings:
            num_of_ratings=re.sub("rating","",num_of_ratings).strip()
            
        if rating is not None:
            rating=rating.strip()
            
        ratings_dict["Rating"]=[rating,num_of_ratings]
        recipe.append(ratings_dict)
        
        time=soup.find_all('li',{"class":"body-copy-small list-item"})
        
        #print(time[0].text)
        
        prep_time=None
        prep_time_dict={}
        minutes=0
        
        if(len(time[0].text.split(':'))>1):
            prep_time=time[0].text.split(':')[1]
        else:
            prep_time=time[0].text.replace("Total time",'')
        
        if prep_time is not None:
            temp=prep_time.split()
            for ind,t in enumerate(temp):
                if t=="hr" or t=="hrs":
                    minutes+=60*int(temp[ind-1])
                if t=="min" or t=="mins":
                    minutes+=int(temp[ind-1])
            prep_time=minutes
            
        prep_time_dict["Prepartion Time"]=prep_time
        recipe.append(prep_time_dict)
        
        cook_time=None
        cook_time_dict={}
        minutes=0
        
        if len(time)>1:
            #print(time[1].text)
            cook_time=time[1].text.split(':')[1]
            temp=cook_time.split()
            for ind,t in enumerate(temp):
                if t=="hr" or t=="hrs":
                    minutes=60*int(temp[ind-1])
                if t=="min" or t=="mins":
                    minutes=minutes+int(temp[ind-1])
            cook_time=minutes
        else:
            cook_time=None
        
        cook_time_dict["Cooking Time"]=cook_time
        recipe.append(cook_time_dict)
        
        recipe_features=soup.find_all('div',{"class":"icon-with-text__children"})
        
        '''
        if len(recipe_features)>1:
            difficulty=recipe_features[1].text
            difficulty_dict={}
            difficulty_dict["Difficulty"]=difficulty
            recipe.append(difficulty_dict)
        '''
    
        if len(recipe_features)>2:
            serves=recipe_features[2].text
            serves=re.sub("Serves","",serves).strip()
        
        serves_dict={}
        serves_dict["Serves"]=serves
        recipe.append(serves_dict)

        result=soup.find_all('table',{"class":"key-value-blocks hidden-print"})
        
        prev=""
        entity=""
        
        calories=None
        fat=None
        cholesterol=None
        carbs=None
        fiber=None
        protein=None
        
        for ind,row_item in enumerate(result[0].find_all("td")):
            if ind==0:
                entity=re.sub("(\xa0)|(\n)|,","",row_item.text)
            else:
                prev=entity
                entity=re.sub("(\xa0)|(\n)|,","",row_item.text)
                if prev=="kcal":
                    calories=entity
                elif prev=="fat":
                    fat=entity
                elif prev=="saturates":
                    cholesterol=entity
                elif prev=="carbs":
                    carbs=entity
                elif prev=="fibre":
                    fiber=entity
                elif prev=="protein":
                    protein=entity
                    
        calories_dict={}
        calories_dict["Calories"]=calories
        recipe.append(calories_dict)
        
        fat_dict={}
        fat_dict["Fat"]=fat
        recipe.append(fat_dict)
        
        cholesterol_dict={}
        cholesterol_dict["Cholesterol"]=cholesterol
        recipe.append(cholesterol_dict)
        
        carbs_dict={}
        carbs_dict["Carbs"]=carbs
        recipe.append(carbs_dict)
        
        fiber_dict={}
        fiber_dict["Fiber"]=fiber
        recipe.append(fiber_dict)
        
        protein_dict={}
        protein_dict["Protein"]=protein
        recipe.append(protein_dict)
        
        result=soup.find_all('li',{"class":"pb-xxs pt-xxs list-item list-item--separator"}) 

        ingredients_dict={}
        ingredients_list=[] 
        ingredients=[] 

        #print ingredients section contents 
        for ingredient in result:  
           ingredients_list=list(filter(None, re.sub(re.compile('<.*?>'),'\n', str(ingredient)).split('\n'))) 
           string=" ".join([ing for ing in ingredients_list])
           ingredients.append(string)
        
        ingredients_dict["Ingredients"]=ingredients
        recipe.append(ingredients_dict)
        
        cooking_dict={}
        cooking_instructions=[]

        #scrape methods of recipe 
        #html=requests.get(url).text 
        #soup = BeautifulSoup(html,"html.parser") 
        div_method=soup.find_all('div',{"class":"editor-content"}) 

        #print cooking instructions 
        #print("\nCOOKING INSTRUCTIONS\n") 
        #f.write("\nCOOKING INSTRUCTIONS\n\n") 
        for i,step in enumerate(div_method): 
           if(i!=0 and len(step.get_text())!=0): 
               if not step.get_text().split()[0].isupper(): 
                   cooking_instructions.append("{}".format(step.get_text()))
                   #print("{}. {}".format(i,step.get_text())) 
                   #f.write("{}. {}\n".format(i,step.get_text())) 
                   #f.write("\n") recipe_ingredient_dictking_instructions
        cooking_dict["Cooking instructions"]=cooking_instructions
        recipe.append(cooking_dict)
        #print() 
        #print()
        #print(recipe)
        #print("------------------------------------------------")
        
        recipe_ingredient_dict["Recipe {}".format(recipe_count)]=recipe
        final_recipes.append(recipe_ingredient_dict)
        recipe_count+=1

Recipe 1: https://www.bbcgoodfood.com/recipes/breakfast-egg-wraps
Recipe 2: https://www.bbcgoodfood.com/recipes/two-minute-breakfast-smoothie
Recipe 3: https://www.bbcgoodfood.com/recipes/breakfast-burrito
Recipe 4: https://www.bbcgoodfood.com/recipes/breakfast-naans
Recipe 5: https://www.bbcgoodfood.com/recipes/raspberry-almond-oat-breakfast-cookies
Recipe 6: https://www.bbcgoodfood.com/recipes/chorizo-halloumi-breakfast-baguette
Recipe 7: https://www.bbcgoodfood.com/recipes/easy-breakfast-burritos
Recipe 8: https://www.bbcgoodfood.com/recipes/kefir-breakfast-smoothie
Recipe 9: https://www.bbcgoodfood.com/recipes/tropical-breakfast-bars
Recipe 10: https://www.bbcgoodfood.com/recipes/vegan-breakfast-muffins
Recipe 11: https://www.bbcgoodfood.com/recipes/slow-cooker-baked-turkish-breakfast-eggs
Recipe 12: https://www.bbcgoodfood.com/recipes/chia-oat-breakfast-scones-yogurt-and-berries
Recipe 13: https://www.bbcgoodfood.com/recipes/slow-cooker-breakfast-beans
Recipe 14: https://www.bbcgo

Recipe 115: https://www.bbcgoodfood.com/recipes/banana-colada-porridge-topper
Recipe 116: https://www.bbcgoodfood.com/recipes/espresso-martini-pancakes
Recipe 117: https://www.bbcgoodfood.com/recipes/slow-cooker-bio-yogurt
Recipe 118: https://www.bbcgoodfood.com/recipes/bircher-muesli-apple-banana
Recipe 119: https://www.bbcgoodfood.com/recipes/vegan-fry
Recipe 120: https://www.bbcgoodfood.com/recipes/fig-jam
Recipe 121: https://www.bbcgoodfood.com/recipes/blackcurrant-compote
Recipe 122: https://www.bbcgoodfood.com/recipes/full-english-shakshuka
Recipe 123: https://www.bbcgoodfood.com/recipes/poppy-seed-buckwheat-porridge
Recipe 124: https://www.bbcgoodfood.com/recipes/perfect-scrambled-eggs-recipe
Recipe 125: https://www.bbcgoodfood.com/recipes/carrot-cake-overnight-oats
Recipe 126: https://www.bbcgoodfood.com/recipes/avocado-strawberry-smoothie
Recipe 127: https://www.bbcgoodfood.com/recipes/akoori-indian-scrambled-eggs
Recipe 128: https://www.bbcgoodfood.com/recipes/baked-green-egg

Recipe 225: https://www.bbcgoodfood.com/recipes/apple-pear-cherry-compote
Recipe 226: https://www.bbcgoodfood.com/recipes/crunchy-granola-berries-cherries
Recipe 227: https://www.bbcgoodfood.com/recipes/smoothie-cubes
Recipe 228: https://www.bbcgoodfood.com/recipes/apple-linseed-porridge
Recipe 229: https://www.bbcgoodfood.com/recipes/vanilla-poppy-seed-swirl-cake
Recipe 230: https://www.bbcgoodfood.com/recipes/skinny-pepper-tomato-ham-omelette
Recipe 231: https://www.bbcgoodfood.com/recipes/slow-cooked-porridge
Recipe 232: https://www.bbcgoodfood.com/recipes/apple-sultana-porridge
Recipe 233: https://www.bbcgoodfood.com/recipes/cinnamon-berry-granola-bars
Recipe 234: https://www.bbcgoodfood.com/recipes/creamy-yogurt-porridge-apricot-ginger-grapefruit-topping
Recipe 235: https://www.bbcgoodfood.com/recipes/omelette-wedges
Recipe 236: https://www.bbcgoodfood.com/recipes/marmalade-whisky-pain-perdu-apples
Recipe 237: https://www.bbcgoodfood.com/recipes/microwave-kale-chilli-eggs
Recipe 2

Recipe 333: https://www.bbcgoodfood.com/recipes/greek-feta-traybake
Recipe 334: https://www.bbcgoodfood.com/recipes/aubergine-tomato-halloumi-pie
Recipe 335: https://www.bbcgoodfood.com/recipes/sticky-pork-lettuce-wraps
Recipe 336: https://www.bbcgoodfood.com/recipes/roast-tomato-soup-pasta
Recipe 337: https://www.bbcgoodfood.com/recipes/chicken-burrito-bowl
Recipe 338: https://www.bbcgoodfood.com/recipes/roast-pork-crushed-grapes-marsala-juniper
Recipe 339: https://www.bbcgoodfood.com/recipes/baked-banana-porridge
Recipe 340: https://www.bbcgoodfood.com/recipes/baked-falafel-cauliflower-tabbouleh-avocado-pea-feta-smash
Recipe 341: https://www.bbcgoodfood.com/recipes/quinoa-salad-avocado-mayo-and-chicken
Recipe 342: https://www.bbcgoodfood.com/recipes/lime-prawn-cocktail-pitta-salad
Recipe 343: https://www.bbcgoodfood.com/recipes/pepper-lemon-spaghetti-basil-pine-nuts
Recipe 344: https://www.bbcgoodfood.com/recipes/smoked-salmon-tartines
Recipe 345: https://www.bbcgoodfood.com/recipes/

Recipe 440: https://www.bbcgoodfood.com/recipes/broccoli-pasta-salad-eggs-sunflower-seeds
Recipe 441: https://www.bbcgoodfood.com/recipes/tomato-burrito-rice
Recipe 442: https://www.bbcgoodfood.com/recipes/barbecued-bavette-steak-tomato-salad
Recipe 443: https://www.bbcgoodfood.com/recipes/summer-pistou
Recipe 444: https://www.bbcgoodfood.com/recipes/chargrilled-chicken-kale-caesar-salad
Recipe 445: https://www.bbcgoodfood.com/recipes/easy-falafels
Recipe 446: https://www.bbcgoodfood.com/recipes/garden-salmon-salad
Recipe 447: https://www.bbcgoodfood.com/recipes/toddler-recipe-cauliflower-cheese-cakes
Recipe 448: https://www.bbcgoodfood.com/recipes/peruvian-toasted-sweetcorn-avocado-quinoa-salad
Recipe 449: https://www.bbcgoodfood.com/recipes/beautiful-beetroot-feta-patties
Recipe 450: https://www.bbcgoodfood.com/recipes/caramelised-honey-carrots
Recipe 451: https://www.bbcgoodfood.com/recipes/bacon-banana-stuffed-french-toast-croissants
Recipe 452: https://www.bbcgoodfood.com/recipes/

...

Recipe 864: https://www.bbcgoodfood.com/recipes/pulled-pork-tacos-pineapple-salsa
Recipe 865: https://www.bbcgoodfood.com/recipes/chocolate-hazelnut-ice-cream-cheesecake
Recipe 866: https://www.bbcgoodfood.com/recipes/creamy-pork-pear-cassoulet
Recipe 867: https://www.bbcgoodfood.com/recipes/smoky-hake-beans-greens
Recipe 868: https://www.bbcgoodfood.com/recipes/spanish-chicken-traybake-chorizo-peppers
Recipe 869: https://www.bbcgoodfood.com/recipes/baked-sea-bass-lemon-caper-dressing
Recipe 870: https://www.bbcgoodfood.com/recipes/squidgy-chocolate-pear-pudding
Recipe 871: https://www.bbcgoodfood.com/recipes/lemon-blueberry-rice-pudding-tart
Recipe 872: https://www.bbcgoodfood.com/recipes/vegan-bolognese
Recipe 873: https://www.bbcgoodfood.com/recipes/cumin-spiced-halloumi-corn-tomato-slaw
Recipe 874: https://www.bbcgoodfood.com/recipes/next-level-dauphinoise-potatoes
Recipe 875: https://www.bbcgoodfood.com/recipes/carrot-ginger-soup
Recipe 876: https://www.bbcgoodfood.com/recipes/veg

Recipe 972: https://www.bbcgoodfood.com/recipes/plain-waffles
Recipe 973: https://www.bbcgoodfood.com/recipes/easy-salmon-coulibiac
Recipe 974: https://www.bbcgoodfood.com/recipes/orange-marmalade-glazed-roast-duck
Recipe 975: https://www.bbcgoodfood.com/recipes/oregano-chicken-squash-traybake
Recipe 976: https://www.bbcgoodfood.com/recipes/spiced-lamb-pie
Recipe 977: https://www.bbcgoodfood.com/recipes/tomato-burrito-rice
Recipe 978: https://www.bbcgoodfood.com/recipes/strawberry-chocolate-marshmallow-dippers
Recipe 979: https://www.bbcgoodfood.com/recipes/katsu-curry-sauce
Recipe 980: https://www.bbcgoodfood.com/recipes/butternut-bacon-fusilli
Recipe 981: https://www.bbcgoodfood.com/recipes/parma-pork-potato-salad
Recipe 982: https://www.bbcgoodfood.com/recipes/luxe-fish-pie
Recipe 983: https://www.bbcgoodfood.com/recipes/samosa-pie
Recipe 984: https://www.bbcgoodfood.com/recipes/toddler-recipe-cauliflower-cheese-cakes
Recipe 985: https://www.bbcgoodfood.com/recipes/salmon-spinach-ta

Recipe 1078: https://www.bbcgoodfood.com/recipes/honey-mustard-creme-fraiche-baked-chicken
Recipe 1079: https://www.bbcgoodfood.com/recipes/smoked-mackerel-pate-cucumber
Recipe 1080: https://www.bbcgoodfood.com/recipes/mushroom-chestnut-rotolo
Recipe 1081: https://www.bbcgoodfood.com/recipes/cider-braised-cabbage-wedges
Recipe 1082: https://www.bbcgoodfood.com/recipes/red-mullet-saffron-baked-orzo-broad-beans
Recipe 1083: https://www.bbcgoodfood.com/recipes/pork-gorgonzola-garlic-butter-pizza
Recipe 1084: https://www.bbcgoodfood.com/recipes/sea-bass-wild-garlic-mayonnaise
Recipe 1085: https://www.bbcgoodfood.com/recipes/free-form-asparagus-potato-tart
Recipe 1086: https://www.bbcgoodfood.com/recipes/wholewheat-spaghetti-avocado-sauce
Recipe 1087: https://www.bbcgoodfood.com/recipes/grilled-mackerel-pickled-gooseberry-ketchup
Recipe 1088: https://www.bbcgoodfood.com/recipes/gooseberry-creme-brulee-tart
Recipe 1089: https://www.bbcgoodfood.com/recipes/quick-pickled-cabbage
Recipe 1090: h

Recipe 1180: https://www.bbcgoodfood.com/recipes/rhubarb-ripple-semi-freddo
Recipe 1181: https://www.bbcgoodfood.com/recipes/artichoke-olive-calzones
Recipe 1182: https://www.bbcgoodfood.com/recipes/grilled-sea-trout-raw-pea-salsa
Recipe 1183: https://www.bbcgoodfood.com/recipes/thai-red-duck-sticky-pineapple-rice
Recipe 1184: https://www.bbcgoodfood.com/recipes/summer-pork-fennel-beans
Recipe 1185: https://www.bbcgoodfood.com/recipes/pumpkin-curry-chickpeas
Recipe 1186: https://www.bbcgoodfood.com/recipes/pineapple-ginger-pavlova
Recipe 1187: https://www.bbcgoodfood.com/recipes/courgette-couscous-salad-tahini-dressing
Recipe 1188: https://www.bbcgoodfood.com/recipes/creamy-cod-chowder-stew
Recipe 1189: https://www.bbcgoodfood.com/recipes/summer-beans-toast-prosciutto
Recipe 1190: https://www.bbcgoodfood.com/recipes/chocolate-fondants
Recipe 1191: https://www.bbcgoodfood.com/recipes/ginger-beef-lettuce-wraps
Recipe 1192: https://www.bbcgoodfood.com/recipes/chocolate-rose-tart
Recipe 11

Recipe 1288: https://www.bbcgoodfood.com/recipes/hake-stewed-peppers
Recipe 1289: https://www.bbcgoodfood.com/recipes/parmesan-roasted-potatoes
Recipe 1290: https://www.bbcgoodfood.com/recipes/chicken-souvlaki
Recipe 1291: https://www.bbcgoodfood.com/recipes/horseradish-yorkshires
Recipe 1292: https://www.bbcgoodfood.com/recipes/chocolate-mint-chilli-truffles
Recipe 1293: https://www.bbcgoodfood.com/recipes/baked-garlic-prawns-with-roasted-peppers-gremolata-toasts
Recipe 1294: https://www.bbcgoodfood.com/recipes/rack-venison-roasted-carrots-forager-sauce
Recipe 1295: https://www.bbcgoodfood.com/recipes/espresso-martini-cheesecake
Recipe 1296: https://www.bbcgoodfood.com/recipes/chocolate-caramel-terrine
Recipe 1297: https://www.bbcgoodfood.com/recipes/jam-roly-poly
Recipe 1298: https://www.bbcgoodfood.com/recipes/jewelled-squash-chestnut-mushroom-wreath
Recipe 1299: https://www.bbcgoodfood.com/recipes/roasted-celeriac-carrots-with-kale-hazelnuts
Recipe 1300: https://www.bbcgoodfood.com

In [ ]:
with open('bbc_food_recipe.json', 'w') as fd:
    fd.write(json.dumps(final_recipes)) 

In [ ]:
meals=["Breakfast","Lunch","Dinner"]
pages=100
for i,meal in enumerate(meals):
    url_list=[]
    for j in range(pages):
        html=requests.get("https://recipes.sparkpeople.com/browse-results.asp?category={}&sort=number_of_ratings&pagenum={}"
                          .format(meal,j+1)).text
        soup = BeautifulSoup(html,"html.parser")
        result=soup.find_all('a',{"class":"item"})
        for k in range(0,len(result)): 
           print(meal,j+1,k+1,result[k].text)
           url_list.append("https://recipes.sparkpeople.com/{}".format(result[k]['href']))
    with open('spark_people_{}_url_list.txt'.format(meal), 'w') as f: 
       for item in list(set(url_list)): 
           f.write("%s\n" % item)
    f.close()

Breakfast 1 1 Applesauce Oatmeal Muffins
Breakfast 1 2 Feta & Spinach Egg Whites
Breakfast 1 3 100-Calorie Cinnamon Pancakes!
Breakfast 1 4 Baked Egg Cups
Breakfast 1 5 Coach Nicole's Mini Vegetable Frittatas
Breakfast 1 6  Flaxseed, Wheat, and Bran Muffins
Breakfast 1 7 Low-Calorie Banana Muffins
Breakfast 1 8 Eggs-n-Oats
Breakfast 1 9 Coach Dean's Blue Oatmeal
Breakfast 1 10 Whole Wheat Waffles
Breakfast 1 11 Low-Fat Dark Chocolate Muffins
Breakfast 1 12 Blueberry Flax Seed Muffins
Breakfast 1 13 Oatmeal Pancakes
Breakfast 1 14 Banana Peanut Butter Oatmeal
Breakfast 1 15 One-Minute Microwave Muffin
Breakfast 1 16 Whole Wheat Pumpkin Spice Pancakes
Breakfast 1 17 Summer No-Cook Oatmeal
Breakfast 1 18 Cottage Cheese Oatmeal Pancakes
Breakfast 1 19 Banana Bread Pancakes
Breakfast 1 20 100-Calorie Carrot Ginger Muffins
Breakfast 1 21 Light Lemon Blueberry Donuts
Breakfast 1 22 Blueberry Flax Microwave Muffin
Breakfast 1 23 Low-Fat Whole Wheat Pancakes
Breakfast 1 24 Egg in the Basket
Bre

Breakfast 6 1 Slow Cooker Apple Pecan Blondie Oatmeal
Breakfast 6 2 Loaded Breakfast Muffins
Breakfast 6 3 Crustless Low-Carb Spinach Quiche
Breakfast 6 4 Crustless Broccoli and Cheese Quiche
Breakfast 6 5 Banana Egg Pancakes
Breakfast 6 6 Whole wheat buttermilk drop biscuits (dough)
Breakfast 6 7 Strawberry Banana Muffins
Breakfast 6 8 Low Carb Cranberry Muffins
Breakfast 6 9 Protein Crepes
Breakfast 6 10 Blueberry Applesauce Oatmeal Muffins
Breakfast 6 11 Pumpkin Oat Muffin/Bars
Breakfast 6 12 Easy Ham & Cheese Quiche
Breakfast 6 13 Whole-Wheat Apple Cinnamon Muffins
Breakfast 6 14 Slow Cooker Banana Hazelnut Torte Oatmeal
Breakfast 6 15 Whole Wheat Pancake and Waffle Batter
Breakfast 6 16 Pumpkin Spice Oatmeal
Breakfast 6 17 Bran Muffins
Breakfast 6 18 Chocolate chip biscotti
Breakfast 6 19 Breakfast Smoothie
Breakfast 6 20 Granny's Zucchini Bread
Breakfast 6 21 Un-Chained Recipe Contest - Healthy Egg Muffin Breakfast Sandwich
Breakfast 6 22 Waffles, Whole Grain
Breakfast 6 23 peanu

Breakfast 11 1 Raisin Bran Muffins
Breakfast 11 2 Mama Love's Lighter Cinnamon Rolls
Breakfast 11 3 Whole wheat/flax pancakes
Breakfast 11 4 CHEAP, FAST & EASY - Egg White Breakfast Sandwich
Breakfast 11 5 Collard Green Pineapple Smoothie
Breakfast 11 6 Cheezy Sausage Muffins
Breakfast 11 7 Turkey Sausage Gravy & Biscuits
Breakfast 11 8 Pumpkin Pie Cream of Wheat Cereal
Breakfast 11 9 Fiber One Parfait 
Breakfast 11 10 Scrambled Eggs with Black Beans and Salsa
Breakfast 11 11 Allie's Zucchini Fritters
Breakfast 11 12 Low Fat Blueberry Yogurt Muffins
Breakfast 11 13 Slow Cooker Steel-Cut Pumpkin Oats
Breakfast 11 14 Farmers Breakfast
Breakfast 11 15 Banana Peanut Butter Smoothie
Breakfast 11 16 Oatmeal Pancakes
Breakfast 11 17 Jenny's Breakfast Burrito
Breakfast 11 18 Granola-Pumpkin Parfait
Breakfast 11 19 Banana Bread
Breakfast 11 20 Apple Cinnamon Oatmeal Pancakes
Breakfast 11 21 Blueberry Banana Muffins
Breakfast 11 22 L. R. Pumpkin muffins with flax and oat bran
Breakfast 11 23 Per

Breakfast 16 1 Delicious Egg Muffins
Breakfast 16 2 Strawberry Oatmeal Smoothie
Breakfast 16 3 Cranberry Apple bread
Breakfast 16 4 Carrot Muffin
Breakfast 16 5 Strawberry Spinach Smoothies
Breakfast 16 6 Protein Pancakes
Breakfast 16 7 Breakfast Quinoa with Blueberries
Breakfast 16 8 Whole Wheat Oatmeal Pancakes
Breakfast 16 9 L.R. Baked egg puff
Breakfast 16 10 ham and egg casserole
Breakfast 16 11 Apple and Cottage Cheese Bake
Breakfast 16 12 Egg Bagels
Breakfast 16 13 Apples & Cinnamon Oatmeal
Breakfast 16 14 Slow Cooker Banana Bread
Breakfast 16 15 Breakfast on the Go 
Breakfast 16 16 Egg in a Hole
Breakfast 16 17 Whole Wheat Pumpkin Scones
Breakfast 16 18 Anything Goes Granola
Breakfast 16 19 Spinach Feta Scramble
Breakfast 16 20 Oat and Bean Waffles
Breakfast 16 21 Breakfast Casserole
Breakfast 16 22 TVP "Oatmeal"
Breakfast 16 23 Mung Sprout and Puffed Rice Salad
Breakfast 16 24 Tex Mex Tofu Scramble
Breakfast 16 25 Southern "Buttermilk" Biscuits & Turkey Sausage Gravy
Breakfast

Breakfast 21 1 High protein, fiber and low fat Breakfast Burrito
Breakfast 21 2 Ground Beef Omelette
Breakfast 21 3 Vegan Good Morning Muffins
Breakfast 21 4 South Beach Friendly Kidney Bean Mash
Breakfast 21 5 Cream Cheese Breakfast Casserole
Breakfast 21 6 homemade pancakes
Breakfast 21 7 Pumpkin pancakes
Breakfast 21 8 Banana Yoghurt Smoothie
Breakfast 21 9 Low-Cal Vegan Pancakes for One
Breakfast 21 10 Banana Muffin (with Yogurt)
Breakfast 21 11 Gramflour pancakes
Breakfast 21 12 Crepes in orange Sauce
Breakfast 21 13 Strawberry Breakfast Shake
Breakfast 21 14 Microwave Poached Egg
Breakfast 21 15 Whole Wheat Banana Bread
Breakfast 21 16 Egg Muffin Breakfast
Breakfast 21 17 Low Fat Whole Wheat Waffles!!!
Breakfast 21 18 Bannana Bread
Breakfast 21 19 cinnamon quinoa with walnuts
Breakfast 21 20 Breakfast Bread Pudding
Breakfast 21 21 Libbi's Breakfast Cassarole
Breakfast 21 22 Incredible Banana Bread
Breakfast 21 23 Homemade Slow Cooker Yogurt
Breakfast 21 24 Quick Gluten-Free Bread

...

Dinner 76 1 Italian Wedding Soup
Dinner 76 2 Garlic and Rosemary Pork Loin Roast
Dinner 76 3 Mike's Mom's Homemade Pizza
Dinner 76 4 PJ's Slow Cooker Pork Loin
Dinner 76 5 Bisquick Impossibly Easy Cheeseburger Pie
Dinner 76 6 Foil Dinner
Dinner 76 7 Bean & Tomato Tortilla Crunch
Dinner 76 8 Fall off the bone beef ribs
Dinner 76 9 Zesty Shrimp and Pasta
Dinner 76 10 Asparagus Favorito
Dinner 76 11 Tortilla Soup
Dinner 76 12 Baked Haddock in Milk
Dinner 76 13 shrimp pasta pesto
Dinner 76 14 Crockpot Chicken & Dumplings
Dinner 76 15 Baked Salmon Dijon
Dinner 76 16 Garlic Twice-Baked Potatoes
Dinner 76 17 Pizza Pinwheels
Dinner 76 18 Porkchop and cheese potato casserole
Dinner 76 19 Taco Hot Dish
Dinner 76 20 Pork Chop and Stuffing casserole
Dinner 76 21 Swiss steak
Dinner 76 22 orange scented quinoa with turkey, peacans & cranberries
Dinner 76 23 Shrimp Alfredo
Dinner 76 24 Beef Teriyaki Stirfry
Dinner 76 25 Chelle Chicken (Chicken in Red Wine)
Dinner 76 26 roasted vegetables
Dinner 76 27

Dinner 81 1 Chili Bean Burgers!
Dinner 81 2 Sharon's Creamy Chicken Noodle Soup
Dinner 81 3 Best Enchiladas
Dinner 81 4 yorkshire pudding
Dinner 81 5 Sweet and Spicy Lentil Chili
Dinner 81 6 Homestyle Turkey Meatloaf
Dinner 81 7 Nancy's Crock Pot Chicken & Noodles 
Dinner 81 8 Crunchy Onion Meatloaf
Dinner 81 9 Slow Cooker Pot Roast with Mushroom Gravy
Dinner 81 10 TM's Liver & Onions
Dinner 81 11 Slow Cooker/Crock Pot Swiss Steak
Dinner 81 12 E-Z Campfire Salmon
Dinner 81 13 creamy zuchini and orzo
Dinner 81 14 Turkey Enchilada Casserole
Dinner 81 15 Chicken Fried Steak
Dinner 81 16 Chicken Burritos
Dinner 81 17 Pork Tenderloin with Chinese Five-Spice
Dinner 81 18 Indian Fry Bread
Dinner 81 19 Salmon Croquettes, Inspired by Alton Brown
Dinner 81 20 Simple Savory Spanish Rice
Dinner 81 21 Jenny's Chicken W/White wine Sauce
Dinner 81 22 Chicken Tostada
Dinner 81 23 Homemade Pizza
Dinner 81 24 Chicken Caprese with Tomato-Basil Cream
Dinner 81 25 Cheese Pizza 
Dinner 81 26 Becky's General

Dinner 86 1 Southwest Ground Turkey Casserole
Dinner 86 2 Super Tofu Stir Fry
Dinner 86 3 30 min meatloaf
Dinner 86 4 Mama's Chicken and Dumplings
Dinner 86 5 Kielbasa & Fried Cabbage
Dinner 86 6 Chicken Noodle Casserole (Tuna Substitute)
Dinner 86 7 Low-Carb Turkey Meatloaf
Dinner 86 8 Vegan Diet BBQ Tofu & Veggies (Peppers, Onions, Tomatoes, & Mushrooms)
Dinner 86 9 Ranch turkey burgers
Dinner 86 10 Fresh spinach and white beans
Dinner 86 11 Oven-fried Chicken Tenders
Dinner 86 12 Garlic Shrimp Pasta
Dinner 86 13 Tator tot hotdish
Dinner 86 14 Ginger Pepper Steak
Dinner 86 15 Beef Brisket
Dinner 86 16 Ground lamb and Chick Peas
Dinner 86 17 Slow cooker Veggie Stew
Dinner 86 18 Stir-fried Tofu with Vegetables
Dinner 86 19 Broccoli-Cheddar Stuffed Meatloaf
Dinner 86 20 Braised Garlic Chicken w/carrots and spinach
Dinner 86 21 Chicken, Peppers, Onions and Tomatoes
Dinner 86 22 Pork Tenderloin
Dinner 86 23 Parmesan Tilapia
Dinner 86 24 Coke Zero Chicken
Dinner 86 25 Mexican Baked Cod
Din

Dinner 91 1 Sweet Orange Salmon
Dinner 91 2 Jewish Challah Shabbos Bread (parve)
Dinner 91 3 Seafood Pasta Primavera
Dinner 91 4 Weight watchers Taco soup
Dinner 91 5 Lean Shepherds Pie
Dinner 91 6 Tapipca pudding
Dinner 91 7 Spicy Low Fat Turkey Chili with Beans
Dinner 91 8 Awesome Marinated Kabobs
Dinner 91 9 Mongolian Chicken
Dinner 91 10 Sausage and Black Bean Dinner
Dinner 91 11 Rustic Lemon-Onion Chicken
Dinner 91 12 Sesame Noodles
Dinner 91 13 Marinated Ginger & Soy Beef
Dinner 91 14 Chicken Fajitas
Dinner 91 15 Shephard's Pie
Dinner 91 16 Ricotta Gnocchi
Dinner 91 17 Mediterranean Chicken Thighs
Dinner 91 18 Pork fajita
Dinner 91 19 Zigeunerschnitzel / Gypsy Schnitzel
Dinner 91 20 Eggplant Tofu Pasta
Dinner 91 21 grouper athenian
Dinner 91 22 Penne and Chicken with Garlic Cream Sauce
Dinner 91 23 Curried pork tenderloin
Dinner 91 24 Garlic Chicken
Dinner 91 25 Paprika Chicken with Sour Cream Gravy
Dinner 91 26 Pinto Bean Burger - Vegan, Gluten Free
Dinner 91 27 Momritz's Turkey

Dinner 96 1 Crispy Orange Beef
Dinner 96 2 Mediterranian Sausage Soup
Dinner 96 3 Chicken Tikka Masala
Dinner 96 4 25-Minute HUGE Southwestern Chicken Wraps
Dinner 96 5 Glazed Carrots and Sugar Snap Peas
Dinner 96 6 Grilled Corn on the Cob
Dinner 96 7 Grilled Asparagus
Dinner 96 8 Savory Pork Tenderloin
Dinner 96 9 Betty's zucchini oven fries
Dinner 96 10 Beef and Bean Chili
Dinner 96 11 Turkey shepards pie
Dinner 96 12 Roasted Chicken Legs and Potatoes
Dinner 96 13 Marianne's Pecan Chicken
Dinner 96 14 Light Chicken Piccata
Dinner 96 15 Cassie's Oven-Roasted Vegetables
Dinner 96 16 *Lemon Dill Chicken - SB (Phase 1)
Dinner 96 17 Sweet Lemon Salmon
Dinner 96 18 Turkey Enchiladas
Dinner 96 19 Chiliastic While Bean Chili
Dinner 96 20 Chicken Paprikash
Dinner 96 21 My Friends Thai Wraps Modified
Dinner 96 22 Chocolate Chia Seed Pudding
Dinner 96 23 Linguine with Tuna Puttanesca
Dinner 96 24 Crock Pot Cajun Chicken and Shrimp Creole
Dinner 96 25 Breaded Not Fried Panko Crusted Shrimp
Dinne

In [ ]:
meal_url_list={}
for meal in meals:
    with open('spark_people_{}_url_list.txt'.format(meal), 'r') as f: 
       data=f.read()
    url_list=data.split()
    meal_url_list.setdefault(meal,[])
    meal_url_list[meal]=url_list

In [ ]:
final_recipes=[]
recipe_count=1
temp=re.compile(r'[\n\r\t]')
for meal in meals:
    for index,url in enumerate(meal_url_list[meal]): 
        recipe=[]
        recipe_ingredient_dict={}
        recipe_ingredient_dict.setdefault("Recipe {}".format(recipe_count),[]) 
        #recipe_ingredient_dict.setdefault(index+1,[]) 
        #Scrape ingredients of recipe 
        html=requests.get(url).text 
        soup = BeautifulSoup(html,"html.parser") 
        
        name=soup.find_all('h1',{"itemprop":"name"})
        name=name[0].text

        print("Recipe {}: {}".format(recipe_count,name))
        
        recipe_id={}
        recipe_id["Title"]=name
        recipe.append(recipe_id)
        
        url_dict={}
        url_dict["URL"]=url
        recipe.append(url_dict)
        
        meal_dict={}
        meal_dict["Meal"]=meal.lower()
        recipe.append(meal_dict)
        
        rating=soup.find_all('span',{"itemprop":"ratingValue"})
        if len(rating)>0:
            rating=rating[0].text+" out of 5."
        else:
            rating=None

        num_of_rating=None
        num_of_ratings=soup.find_all('span',{"itemprop":"ratingCount"})
        if len(num_of_ratings)>0:
            num_of_ratings=num_of_ratings[0].text
        else:
            num_of_ratings=None
        
        ratings_dict={}
        ratings_dict["Rating"]=[rating,num_of_ratings]
        recipe.append(ratings_dict)
        
        recipe_time_servings=soup.find_all('div',{"class":"prep_box"})
        
        prep_time=None
        cook_time=None
        number_of_servings=None
        prep_time_dict={}
        cook_time_dict={}
        serves_dict={}
        for i in range(len(recipe_time_servings)):
            task,val=recipe_time_servings[i].text.split(':')
            if task=="Minutes to Prepare":
                prep_time=val
            elif task=="Minutes to Cook":
                cook_time=val
            else:
                number_of_servings=val
        
        prep_time_dict["Prepartion Time"]=prep_time
        recipe.append(prep_time_dict)
        
        cook_time_dict["Cooking Time"]=cook_time
        recipe.append(cook_time_dict)
        
        serves_dict["Serves"]=number_of_servings
        recipe.append(serves_dict)
        
        calories_dict={}
        calories=soup.find_all('span',{"itemprop":"calories"})
        if len(calories)>0:
            calories=calories[0].text
        else:
            calories=None
        calories_dict["Calories"]=calories
        recipe.append(calories_dict)

        fat_dict={}
        fat=soup.find_all('span',{"itemprop":"fatContent"})
        if len(fat)>0:
            fat=fat[0].text
        else:
            fat=None
        fat_dict["Fat"]=fat
        recipe.append(fat_dict)

        cholesterol_dict={}
        cholesterol=soup.find_all('span',{"itemprop":"cholesterolContent"})
        if len(cholesterol)>0:
            cholesterol=cholesterol[0].text
        else:
            cholesterol=None
        cholesterol_dict["Cholesterol"]=cholesterol
        recipe.append(cholesterol_dict)

        '''
        sodium_dict={}
        sodium=soup.find_all('span',{"itemprop":"sodiumContent"})
        if len(sodium)>0:
            sodium=sodium[0].text
        else:
            sodium=None
        sodium_dict["Salt"]=sodium
        recipe.append(sodium_dict)
        '''

        carbs_dict={}
        carbs=soup.find_all('span',{"itemprop":"carbohydrateContent"})
        if len(carbs)>0:
            carbs=carbs[0].text
        else:
            carbs=None
        carbs_dict["Carbs"]=carbs
        recipe.append(carbs_dict)

        fiber_dict={}
        fiber=soup.find_all('span',{"itemprop":"fiberContent"})
        if len(fiber)>0:
            fiber=fiber[0].text
        else:
            fiber=None
        fiber_dict["Fiber"]=fiber
        recipe.append(fiber_dict)

        protein_dict={}
        protein=soup.find_all('span',{"itemprop":"proteinContent"})
        if len(protein)>0:
            protein=protein[0].text
        else:
            protein=None
        protein_dict["Protein"]=protein
        recipe.append(protein_dict)
        
        ingredients_dict={}
        ingredients_list=[] 
        ingredients=[] 

        ingredients=soup.find_all('span',{"itemprop":"ingredients"})
        for i in range(len(ingredients)):
            ingredients_list.append(ingredients[i].text)
        
        ingredients_dict["Ingredients"]=ingredients_list
        recipe.append(ingredients_dict)

        cooking_dict={}
        cooking_instructions=[]

        instructions=soup.find_all('div',{"id":"directions_w"})
        t=instructions[0].text.split('.')[:-2]
        cooking_instructions=[temp.sub("", s) for s in t if len(s)!=0]
        
        cooking_dict["Cooking instructions"]=cooking_instructions
        recipe.append(cooking_dict)
      
        recipe_ingredient_dict["Recipe {}".format(recipe_count)]=recipe
        final_recipes.append(recipe_ingredient_dict)
        recipe_count+=1

Recipe 1: Breakfast caserole
Recipe 2: Crockpot Yogurt
Recipe 3: Scrambled Eggs with Black Beans and Salsa
Recipe 4: Flaxseed Morning Glory Muffins
Recipe 5: Strawberry Yogurt Breakfast Split
Recipe 6: Nanna Lynne's  Rainbow Fruit Squares
Recipe 7: Visalus Shake w/almond milk
Recipe 8: Raspberry Oat Bran Muffins
Recipe 9: Grab & Go Cottage Cheese Crunch
Recipe 10: Strawberry Stuffed French Toast w/ Strawberry Syrup
Recipe 11: Low Carb Gluten Free Coconut Flour Waffles
Recipe 12: Blueberry Zucchini Pancakes
Recipe 13: College breakfast
Recipe 14: South Beach Friendly Mock Oatmeal (P1)
Recipe 15: Cottage Cheese Pancakes
Recipe 16: Baked Oatmeal
Recipe 17: Low- Carb,  High- Fiber Bran Muffins (rich in omega 3 fatty acid)
Recipe 18: Japanese Breakfast
Recipe 19: Low-Carb, Gluten-Free Granola
Recipe 20: Misoshiru
Recipe 21: Kristin's Banana Chip Muffins
Recipe 22: Mango-Pineapple-Banana Smoothie
Recipe 23: Nutri Bullet Toxin Cleansing Blast
Recipe 24: Crab & Cheese Omlett
Recipe 25: Butterm

Recipe 208: Fiesta Egg Whites
Recipe 209: Danica's Protein Pancakes
Recipe 210: Oatmeal protein pancakes
Recipe 211: Low-Cal Carrot Banana Muffins
Recipe 212: Oatmeal Breakfast Cookie
Recipe 213: Low-Calorie Banana Muffins
Recipe 214: L. R. Pumpkin muffins with flax and oat bran
Recipe 215: Pancakes GF Protein Raspberry Oat CC 
Recipe 216: Pumpkin Zucchini Bread
Recipe 217: Potato Cakes (mashed potato pancakes)
Recipe 218: Egg & Cheese Breakfast Tacos with Homemade Salsa
Recipe 219: Oatmeal, steel cut, nuts, cranberries, apples, honey. Mare's recipe.
Recipe 220: Basic Pancake Batter
Recipe 221: Smoked Salmon Breakfast Wrap
Recipe 222: L.R. Kashi banana muffins
Recipe 223: Dark Chocolate Banana Oat Muffins
Recipe 224: Crockpot Egg Bake
Recipe 225: Mini Crustless Quiche Cups
Recipe 226: Tuesday night omlets
Recipe 227: Apple Cottage Pancakes
Recipe 228: Zucchini Feta Muffins
Recipe 229: Low-carb Buttermilk Cinnamon Waffles
Recipe 230: The Perfect Fried Egg
Recipe 231: Bran Muffins
Recipe

Recipe 416: Oatmeal Pancakes with Strawberry Sauce
Recipe 417: pumpkin-raisin bread pudding
Recipe 418: Fruity Pancake Bites
Recipe 419: breakfast burritos
Recipe 420: Fruit Breakfast Cake
Recipe 421: Cinnamon Donut Muffins
Recipe 422: Banana Mango Frosty
Recipe 423: Zucchini Carrot Bread
Recipe 424: Smoothie:  Banana, strawberry, yogurt. (Makes 2 2c servings)
Recipe 425: Ezekiel Bread French Toast
Recipe 426: Berry Banana Smoothies
Recipe 427: Ines' Banana Nut Bread, or Muffins
Recipe 428: Baked Banana Bread Oatmeal
Recipe 429: Pumpkin Carrot Muffins
Recipe 430: Baked Oatmeal
Recipe 431: "Sausage", Biscuits & Gravy
Recipe 432: Pumpkin Blueberry Muffins egg-free dairy-free 
Recipe 433: Quinoa Banana Pudding
Recipe 434: Pancake, Sweet Dutch Apple
Recipe 435: Goetta
Recipe 436: Toasted Egg & Cheese Sandwich
Recipe 437: Low Carb Pancake
Recipe 438: One Minute Flax Muffin - Low Carb
Recipe 439: Blueberry Zucchini Muffins
Recipe 440: Maltby Greek Feta Omelete
Recipe 441: Apricot, Orange, Cr

Recipe 629: Oatmeal Blueberry Muffins
Recipe 630: Breakfast Egg Bake
Recipe 631: Vegan Banana Muffins
Recipe 632: Quick & Easy Breakfast Fiber Bars
Recipe 633: Beta Booster Cake
Recipe 634: L.R. chocolate muffins
Recipe 635: Single serving whole wheat french toast
Recipe 636: Easy Ham, Broccoli, Cheddar Quiche
Recipe 637: Breakfast Bagels
Recipe 638: Overnight Steel Cut Oats
Recipe 639: Banana Bread/Muffins
Recipe 640: 100-Calorie Carrot Ginger Muffins
Recipe 641: Peanut butter Banana Breakfast smoothie
Recipe 642: Mocha Muffins with Chocolate Chunks (Low Carb!)
Recipe 643: Ali's breakfast smoothie
Recipe 644: Blueberry Flax Seed Muffins
Recipe 645: Chewy Granola Bars
Recipe 646: Baked French Toast Casserole
Recipe 647: Lean Eggs & Ham
Recipe 648: Raisin Bran Muffins
Recipe 649: Protein Pancakes (Without the Powder!)
Recipe 650: Apple Banana Bread
Recipe 651: Eggy Cheesy Muffin
Recipe 652: Hearty Grain Breakfast
Recipe 653: 200 Calorie-On the go Egg Pockets
Recipe 654: Egg, Toast, and 

Recipe 835: Baked Egg Cups
Recipe 836: Junk Yard Omelet
Recipe 837: Light French Toast
Recipe 838: Spinach & Artichoke Quiche "Muffins" South Beach
Recipe 839: Protein Breakfast Cookie
Recipe 840: Low Fat Rhubarb Blueberry Muffins
Recipe 841: Jenny's PB Strawberry Banana Soy Smoothie
Recipe 842: Omlette to Go
Recipe 843: Fabulous Low-Cal French Toast
Recipe 844: Whole Wheat Pancakes
Recipe 845: Easy Egg Breakfast/Brunch Casserole
Recipe 846: gluten free american style fluffy pancakes 
Recipe 847: Strawberry Oatmeal Smoothie
Recipe 848: Best Granola Bars
Recipe 849: Pumpkin Health-craze Muffins
Recipe 850: Banana Bread Pancakes
Recipe 851: Pumpkin Scones
Recipe 852: Avocado Toast 
Recipe 853: Almond Ring Coffee Cake
Recipe 854: Cheddar Scramble
Recipe 855: Jenny's Breakfast Burrito
Recipe 856: L.R. banana raisin oat bran & flax  muffins
Recipe 857: A Berry Good Smoothie
Recipe 858: Low Sodium Pancakes
Recipe 859: Healthy Raisin Bran Muffins
Recipe 860: Banana Soy Protein Smoothie
Recipe

...

Recipe 7580: Roasted Italian Pork Tenderloin
Recipe 7581: Shira's Chicken and Vegetable Soup
Recipe 7582:  Arroz con gandules
Recipe 7583: Mexican Cornbread Casserole
Recipe 7584: 15-Minute Fish with Parsley Pesto
Recipe 7585: Mexican Halibut Bake
Recipe 7586: Mike's Mom's Homemade Pizza
Recipe 7587: Slow Cooker Bacon & Cheese Potatoes
Recipe 7588: Chili Bean Burgers!
Recipe 7589: Chicken Enchiladas
Recipe 7590: Baked Spaghetti & Meatballs
Recipe 7591: Stuffed Cabbage Rolls
Recipe 7592: Italian Chicken in the Crock pot
Recipe 7593: Bubble Up Pizza Casserole
Recipe 7594: One skillet ground beef and pasta
Recipe 7595: yummy turkey burgers
Recipe 7596: Unchained Recipe Contest Red Pepper Shrimp Pasta made over from Michelle's in Ketchum Idaho
Recipe 7597: Paper Baked Halibut
Recipe 7598: Brazilian Black Bean Crock Pot Dish
Recipe 7599: Saute-Steamed Bok Choy
Recipe 7600: Taco Meat Seasoning
Recipe 7601: Rotini beef casserole
Recipe 7602: Macaroni and Cheese for Adults
Recipe 7603: Slow Co

Recipe 7782: Chicken Drumstick Lollipops with Grilled Lemon Parsley Sauce
Recipe 7783: Southwestern Quesadillas
Recipe 7784: Fettuccine Alfredo with Chicken (1 c)
Recipe 7785: Sharon's Creamy Chicken Noodle Soup
Recipe 7786: Chicken Enchilada-Style for the Slow Cooker
Recipe 7787: Shrimp Pad Thai 
Recipe 7788: Ground Turkey Stove Top Pot Pie - Low Fat
Recipe 7789: Easy Chicken Curry
Recipe 7790: Black Bean Chicken
Recipe 7791: Linda's Italian Zucchini Pot
Recipe 7792: Crunchy Spiced Chicken (Un-Chained Recipe Contest)
Recipe 7793: Yummy Pork Chops
Recipe 7794: Middle Eastern Spinach Lentils with Yogurt
Recipe 7795: Slow Cooker Jambalya
Recipe 7796: Easy Appley Pork Chops
Recipe 7797: Irish Corned Beef and Cabbage
Recipe 7798: Sun-Dried Tomato, Pesto & Mozzarella Turkey Burgers
Recipe 7799: Mexican Casserole
Recipe 7800: Spicy Shrimp-Vodka Sauce with Penne Rigate Pasta 
Recipe 7801: Ginger Beef
Recipe 7802: Wild Baked Salmon
Recipe 7803: Spaghetti  Bake
Recipe 7804: Halibut Stew with Fr

Recipe 7988: Sweetly Succulent Mahi Mahi
Recipe 7989: Creamy Chicken, Bacon and Mushroom
Recipe 7990: Steph2003's Chicken & Dumplings
Recipe 7991: Chicken Korma
Recipe 7992: Slow Cooker Beef Stew 
Recipe 7993: sweet glazed salmon
Recipe 7994: Trout with Cream & Honey Sauce
Recipe 7995: creamy zuchini and orzo
Recipe 7996: Spinach stuffed turkey loaf
Recipe 7997: Ranch Baked Chicken
Recipe 7998: Ratatouille
Recipe 7999: Baked Egg Rolls 
Recipe 8000: Sesame Chicken and Vegetable Stir Fry
Recipe 8001: Yummy Hamburger Pie!
Recipe 8002: Spinach & Butternut Squash Lasagna
Recipe 8003: Slow Cooker Chicken Tikka Masala
Recipe 8004: One-Pan Chicken Parmesan
Recipe 8005: Pineapple Peach Smoothie
Recipe 8006: Beef and Noodles
Recipe 8007: Parmesan Chicken
Recipe 8008: SW Ground Turkey & rice 1 skillet meal
Recipe 8009: Bolognese sauce
Recipe 8010: Stuffed Chicken Breast
Recipe 8011: Venison Shepherds pie
Recipe 8012: bbq biscuit cups
Recipe 8013: Ground Turkey & Potato Casserole
Recipe 8014: Bow 

Recipe 8192: Garlic and Olive Oil Spaghetti Squash
Recipe 8193: Carribean Brown Rice
Recipe 8194: Pita Pizza
Recipe 8195: Lemon Garlic Salmon
Recipe 8196: Paprika Chicken with Sour Cream Gravy
Recipe 8197: Cajun Pork Chops
Recipe 8198: Beef and Bean Enchiladas
Recipe 8199: Balsamic Grilled Chicken Breast
Recipe 8200: Stuffed Portobello Mushrooms on the Grill
Recipe 8201: Shrimp Stir-fry
Recipe 8202: General Tso's chicken
Recipe 8203: Orange Chicken
Recipe 8204: Beef and Leek Stir Fry
Recipe 8205: Ferber's Simple Turkey Shepherd's Pie
Recipe 8206: Easy Turkey & Chicken Meatballs or Meatloaf
Recipe 8207: Crockpot Brown Sugar Chicken
Recipe 8208: Herbed Spinach, Feta, Ricotta stuffed Chicken
Recipe 8209: Parmesean Crusted Tilapia
Recipe 8210: Chicken Cordon Blue
Recipe 8211: Alaskan Salmon patties made by an alaskan woman! 
Recipe 8212: Mom's Meatloaf
Recipe 8213: Grilled Mediterranean Ahi Tuna
Recipe 8214: Chicken Cream Cheese Crescents
Recipe 8215: Bacon Ranch Slow Cooker Chicken
Recipe

Recipe 8397: Slow Cooker Split Pea and Ham Soup
Recipe 8398: Eggplant Pasta
Recipe 8399: Lean Shepherds Pie
Recipe 8400: Low-Carb Turkey Meatloaf
Recipe 8401: Crustless Spinach, Onion and Feta Quiche
Recipe 8402: Sunday chuck roast
Recipe 8403: Salmon Cakes
Recipe 8404: Low-Fat Spinach Lasagna
Recipe 8405: Salisbury Steak
Recipe 8406: Crunchy Walnut Pork
Recipe 8407: Spaghetti Squash Dinner
Recipe 8408: Mexicali Skillet Rice
Recipe 8409: Kapusta (Sausage & Sauerkraut)
Recipe 8410: Parmesan Scallops
Recipe 8411: Honey Chicken Kabobs
Recipe 8412: Southwest Ground Turkey Casserole
Recipe 8413: Katie's Red Beans and Rice
Recipe 8414: Cheesy  Potatoes
Recipe 8415: Marsala Steak with Mushrooms Remix
Recipe 8416: Zack's 'Refried' Black Beans
Recipe 8417:  Low Carb Deep Dish Pizza Quiche Recipe
Recipe 8418: Sloppy Joes
Recipe 8419: Chilie Relleno Casserole
Recipe 8420: Shrimp Scampi
Recipe 8421: Spiced Chicken with yogurt sauce
Recipe 8422: Ferber's, now Famous, Spaghetti Sauce
Recipe 8423: Pi

In [ ]:
with open('spark_people_food_recipe.json', 'w') as fd:
    fd.write(json.dumps(final_recipes)) 